# 全部采用本地化部署的方式和开源模型来建立的问答能力和本地化知识库集成的能力

定义语言模型

In [1]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()
model = model.eval()

/home/xiaodong/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No compiled kernel found.
Compiling kernels : /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.c -shared -o /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.so
Load kernel : /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 6
Using quantization cache
Applying quantization to glm layers


按照langchain标准进行模型封装

In [2]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
from llama_index import LLMPredictor

class CustomLLM(LLM):

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response, history = model.chat(tokenizer, prompt, history=[])
        # only return newly generated tokens
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": "THUDM/chatglm-6b-int4"}

    @property
    def _llm_type(self) -> str:
        return "custom"
    
llm_predictor = LLMPredictor(llm=CustomLLM())



引入embedding模型

In [3]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))

定义包含embedding模型和语言模型的Service Context

In [4]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm_predictor=llm_predictor)

构建本地知识库（本地知识的索引库）

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from llama_index import SimpleDirectoryReader
from llama_index import GPTVectorStoreIndex
from llama_index.node_parser import SimpleNodeParser

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=20)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('./data/faq/').load_data()
nodes = parser.get_nodes_from_documents(documents)


index = GPTVectorStoreIndex(nodes=nodes, service_context=service_context)
query_engine = index.as_query_engine()

Created a chunk of size 130, which is longer than the specified 100


结合本地知识库和语言模型进行提问

In [6]:
question = """
请用中文回答下述问题：

Q: 你们能配送到天津吗？另外，你们的退货政策是怎样的？
"""
response = query_engine.query(question)
print(response)

The dtype of attention mask (torch.int64) is not bool


A: 我们的配送范围是全国大部分省份，包括北京、上海、天津、重庆、河北、山西、辽宁、吉林、黑龙江、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、广东、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏和新疆。因此，我们在天津的配送是可行的。至于退货政策，我们会在订单发货后尽快处理退货，并在“我的订单”页面展示退货说明，请按照说明进行退货。


In [7]:
question = """
请用中文回答下述问题：

Q: 请介绍一下ChatGLM？
"""
response = query_engine.query(question)
print(response)

ChatGLM-6B 是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM-6B 开发的，我的任务是针对用户的问题和要求提供适当的答复和支持。


In [8]:
question = """
请用中文回答下述问题：

Q: 请介绍一下语言模型中的Alpaca。
"""
response = query_engine.query(question)
print(response)

A: Alpaca是一个由清华大学 KEG 实验室和智谱AI开发的语言模型，旨在提供自然语言处理和生成的能力。它使用深度学习技术来训练，并能够生成高质量的文本，例如文章、对话和邮件。Alpaca 还可以进行语言理解和文本生成，因此可以与人类进行对话，并生成各种类型的文本。它的主要优势在于其质量和速度，可以处理大量的文本数据，并且可以自动学习和改进。


In [9]:
question = """
请用中文回答下述问题：

Q: 请写一段python代码，展示如何使用Alpaca的模型在本地GPU运行支持语言任务的能力。
"""
response = query_engine.query(question)
print(response)

以下是一段使用Python代码展示如何使用Alpaca模型在本地GPU运行支持语言任务的能力的步骤：

1. 安装必要的库

首先，需要安装必要的库，包括PyTorch和CUDA。可以使用以下命令安装：

```
pip install torch torchvision
pip install CUDA
```

2. 导入必要的库

导入必要的库，包括PyTorch和CUDA库。可以使用以下代码导入：

```
import torch
import torchvision.models as models
import CUDART
```

3. 加载Alpaca模型

可以使用以下代码加载Alpaca模型：

```
model = CUDART.load('your_paca_model_path', 'cuda')
```

4. 运行GPU训练

使用以下代码运行GPU训练：

```
device = CUDART.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
```

5. 运行本地GPU训练

可以使用以下代码运行本地GPU训练：

```
device = CUDART.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()
```

以上代码展示了如何使用PyTorch和CUDA库来加载Alpaca模型并在本地GPU上运行训练。


使用新的TextSplitter来对原始内容进行处理，换一个新的index来处理索引，对文章进行总结

In [10]:
from langchain.text_splitter import SpacyTextSplitter

new_text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size = 2048)
new_parser = SimpleNodeParser(text_splitter=new_text_splitter)
new_documents = SimpleDirectoryReader('./data/mr_fujino').load_data()
new_nodes = new_parser.get_nodes_from_documents(new_documents)



In [11]:
from llama_index import GPTListIndex

list_index = GPTListIndex(nodes=new_nodes, service_context=service_context)

# response = list_index.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:", response_mode="tree_summarize")

query_engine = list_index.as_query_engine(
    response_mode="tree_summarize"
)
response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (1680 > 1024). Running this sequence through the model will result in indexing errors


这段内容主要是描述鲁迅先生在日本留学期间的经历和对于中国社会、文化、政治等方面的观察和思考。其中包括他在日本仙台医学专门学校学习的经历，与学校的老师们和学生们之间的互动和交往，以及他对中国传统文化和医学知识的热爱和追求。同时，他表达了对藤野先生的感激之情，同时也对他穿衣搭配和学术成就等方面进行了赞扬和评价。

整段内容表达了鲁迅先生对这位老师的感激之情和敬意，也展现了他对于学术和知识的追求和热爱。他描述了他收藏的讲义，以及他对于夜间思考和良心发现的经历。这段内容主要讲述了他在日本留学期间的经历和对于中国社会、文化、政治等方面的观察和思考，是《藤野先生》节选的全文总结。


换用TreeIndex来试试效果

In [12]:
from llama_index import GPTTreeIndex

# define LLM
tree_index = GPTTreeIndex(nodes=new_nodes, service_context=service_context)
# response = tree_index.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:", mode="summarize")

#query_engine = tree_index.as_query_engine(
#    response_mode="summarize"
#)

query_engine = tree_index.as_query_engine()

response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)

以下是新的总结：

在这段文字中，鲁迅先生描述了自己在日本医学专门学校学习的经历，以及与藤野先生相处的时光。

自己最初来到仙台，被学校收为学生，住在监狱旁边的客店里，蚊子太多，饭食不好，但好处是学校不要学费，还提供住宿和饮食。后来，学生潮中有中国留学生，于是去研究他们的文化和生活方式。

在学生中，有一位藤野先生，他的衣服很时髦，但会忘记带领结，有时也会和学生们一起跳舞。他讲述了医学解剖的历史，强调了中国的医学翻译和研究，也提到了他的学生中的一些有趣人物。

最后，藤野先生将自己的讲义修改过，并且每星期都要送自己看一次，让自己对医学知识有了更深入的了解。

在学年试验之后，鲁迅先生去了东京玩一夏天，秋初再回学校，成绩早已发表了，同学一百余人之中，自己在中间，不过是没有落第。这回藤野先生所担任的功课，是解剖实习和局部解剖学。

解剖实习了大概一星期，藤野先生又叫我去了，很高兴地，仍用了极有抑扬的声调对我说道：——
“我因为听说中国人是很敬重鬼的，所以很担心，怕你不肯解剖尸体。

现在总算放心了，没有这回事。”

但他也偶有使我很为难的时候。

他听说中国的女人是裹脚的，但不知道详细，所以要问我怎么裹法，足骨变成怎样的畸形，还叹息道，“总要看一看才知道。

究竟是怎么一回事呢？”

有一天，本级的学生会干事到我寓里来了，要借我的讲义看。

我检出来交给他们，却只翻检了一通，并没有带走。

但他们一走，邮差就送到一封很厚的信，打开一看，第一句是：——
“你改悔罢！”
这是《新约》上的句子罢，但经托尔斯泰新近引用过的。


尝试Vector Store - pinecone

In [13]:
'''
# 目前这个不分不工作
import pinecone
from llama_index import StorageContext
from llama_index.vector_stores import PineconeVectorStore
import os

# pinecone_api_key = os.environ.get("PINECONE_API_KEY")
# pinecone_env = os.environ.get("PINECONE_ENV")

pinecone_api_key = os.environ['PINECONE_API_KEY']
pinecone_env = os.environ['PINECONE_ENV']


# init pinecone
pinecone.init(pinecone_api_key, pinecone_env)
pinecone.create_index("xdtest1", dimension=1536, metric="cosine", pod_type="p1") # pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")

# construct vector store and customize storage context
storage_context = StorageContext.from_defaults(
    vector_store = PineconeVectorStore(pinecone.Index("xdtest1"))
)

# Load documents and build index
documents_p = SimpleDirectoryReader('./data/mr_fujino').load_data()
index_p = GPTVectorStoreIndex.from_documents(documents_p, service_context=service_context, storage_context=storage_context)

query_engine = index_p.as_query_engine()

response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)
'''

'\n# 目前这个不分不工作\nimport pinecone\nfrom llama_index import StorageContext\nfrom llama_index.vector_stores import PineconeVectorStore\nimport os\n\n# pinecone_api_key = os.environ.get("PINECONE_API_KEY")\n# pinecone_env = os.environ.get("PINECONE_ENV")\n\npinecone_api_key = os.environ[\'PINECONE_API_KEY\']\npinecone_env = os.environ[\'PINECONE_ENV\']\n\n\n# init pinecone\npinecone.init(pinecone_api_key, pinecone_env)\npinecone.create_index("xdtest1", dimension=1536, metric="cosine", pod_type="p1") # pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")\n\n# construct vector store and customize storage context\nstorage_context = StorageContext.from_defaults(\n    vector_store = PineconeVectorStore(pinecone.Index("xdtest1"))\n)\n\n# Load documents and build index\ndocuments_p = SimpleDirectoryReader(\'./data/mr_fujino\').load_data()\nindex_p = GPTVectorStoreIndex.from_documents(documents_p, service_context=service_context, storage_context=storage_context)\n\

Faiss Vector Store的使用

In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [18]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredFileLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

em_model = "GanymedeNil/text2vec-large-chinese"
em_device = "cuda"

hgf_embeddings = HuggingFaceEmbeddings(model_name=em_model,
                                        model_kwargs={'device': em_device})

import re

# 文本分句长度
SENTENCE_SIZE = 100

class ChineseTextSplitter(CharacterTextSplitter):
    def __init__(self, pdf: bool = False, sentence_size: int = SENTENCE_SIZE, **kwargs):
        super().__init__(**kwargs)
        self.pdf = pdf
        self.sentence_size = sentence_size

    def split_text1(self, text: str) -> List[str]:
        if self.pdf:
            text = re.sub(r"\n{3,}", "\n", text)
            text = re.sub('\s', ' ', text)
            text = text.replace("\n\n", "")
        sent_sep_pattern = re.compile('([﹒﹔﹖﹗．。！？]["’”」』]{0,2}|(?=["‘“「『]{1,2}|$))')  # del ：；
        sent_list = []
        for ele in sent_sep_pattern.split(text):
            if sent_sep_pattern.match(ele) and sent_list:
                sent_list[-1] += ele
            elif ele:
                sent_list.append(ele)
        return sent_list

    def split_text(self, text: str) -> List[str]:   ##此处需要进一步优化逻辑
        if self.pdf:
            text = re.sub(r"\n{3,}", r"\n", text)
            text = re.sub('\s', " ", text)
            text = re.sub("\n\n", "", text)

        text = re.sub(r'([;；.!?。！？\?])([^”’])', r"\1\n\2", text)  # 单字符断句符
        text = re.sub(r'(\.{6})([^"’”」』])', r"\1\n\2", text)  # 英文省略号
        text = re.sub(r'(\…{2})([^"’”」』])', r"\1\n\2", text)  # 中文省略号
        text = re.sub(r'([;；!?。！？\?]["’”」』]{0,2})([^;；!?，。！？\?])', r'\1\n\2', text)
        # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
        text = text.rstrip()  # 段尾如果有多余的\n就去掉它
        # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
        ls = [i for i in text.split("\n") if i]
        for ele in ls:
            if len(ele) > self.sentence_size:
                ele1 = re.sub(r'([,，.]["’”」』]{0,2})([^,，.])', r'\1\n\2', ele)
                ele1_ls = ele1.split("\n")
                for ele_ele1 in ele1_ls:
                    if len(ele_ele1) > self.sentence_size:
                        ele_ele2 = re.sub(r'([\n]{1,}| {2,}["’”」』]{0,2})([^\s])', r'\1\n\2', ele_ele1)
                        ele2_ls = ele_ele2.split("\n")
                        for ele_ele2 in ele2_ls:
                            if len(ele_ele2) > self.sentence_size:
                                ele_ele3 = re.sub('( ["’”」』]{0,2})([^ ])', r'\1\n\2', ele_ele2)
                                ele2_id = ele2_ls.index(ele_ele2)
                                ele2_ls = ele2_ls[:ele2_id] + [i for i in ele_ele3.split("\n") if i] + ele2_ls[
                                                                                                       ele2_id + 1:]
                        ele_id = ele1_ls.index(ele_ele1)
                        ele1_ls = ele1_ls[:ele_id] + [i for i in ele2_ls if i] + ele1_ls[ele_id + 1:]

                id = ls.index(ele)
                ls = ls[:id] + [i for i in ele1_ls if i] + ls[id + 1:]
        return ls

filepath = "data/faq/ecommerce_faq.txt"
loader = TextLoader(filepath)
textsplitter = ChineseTextSplitter(pdf=False, sentence_size=SENTENCE_SIZE)
docs = loader.load_and_split(textsplitter)

vs_path = "./vector_storage"
faiss_vector_store = FAISS.from_documents(docs, hgf_embeddings)


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: GanymedeNil/text2vec-large-chinese
Load pretrained SentenceTransformer: GanymedeNil/text2vec-large-chinese
No sentence-transformers model found with name /home/xiaodong/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


Batches: 100%|██████████| 3/3 [00:00<00:00,  6.82it/s]


In [19]:
faiss_vector_store.save_local(vs_path)

faiss_vector_store = FAISS.load_local(vs_path, hgf_embeddings)
query_faiss = "配送范围是？"
result_docs = faiss_vector_store.similarity_search(query=query_faiss)
print(result_docs[0].page_content)

Batches: 100%|██████████| 1/1 [00:00<00:00, 104.95it/s]

Q: 支持哪些省份配送？


In [24]:
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore
from llama_index import StorageContext, load_index_from_storage

new_text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size = 2048)
new_parser = SimpleNodeParser(text_splitter=new_text_splitter)
new_documents = SimpleDirectoryReader('./data/mr_fujino').load_data()
new_nodes = new_parser.get_nodes_from_documents(new_documents)

dimention_faiss = 1536
faiss_index = faiss.IndexFlatL2(dimention_faiss)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults( docstore=SimpleDocumentStore(), vector_store=vector_store, index_store=SimpleIndexStore())
storage_context.docstore.add_documents(new_nodes)
index_by_faiss = GPTVectorStoreIndex(new_nodes, storage_context=storage_context, service_context=service_context)

index_by_faiss.storage_context.persist()


Batches: 100%|██████████| 1/1 [00:00<00:00, 38.11it/s]


AssertionError: 

In [ ]:
# load index from disk
vector_store_loaded = FaissVectorStore.from_persist_dir('./storage')
# 下面这个部分必须要提供persist_dir这个路径，似乎缺省不起作用
storage_context_loaded = StorageContext.from_defaults(vector_store=vector_store_loaded, persist_dir="./storage")
index_by_faiss_loaded = load_index_from_storage(storage_context=storage_context_loaded)

query_engine = index_by_faiss_loaded.as_query_engine()
response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)

INFO:root:Loading llama_index.vector_stores.faiss from ./storage/vector_store.json.
Loading llama_index.vector_stores.faiss from ./storage/vector_store.json.
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 57 tokens
> [retrieve] Total embedding token usage: 57 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2220 tokens
> [get_response] Total LLM token usage: 2220 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens

鲁迅先生回忆了他在仙台的经历，他曾经被藤野先生教导，但最终决定不学医学，而是去学生物学。藤野先生有些悲哀，但他叮嘱鲁迅先生将来照相并及时通信告诉他此后的状况。然而，鲁迅先生多年没有照相，也没有写信，从藤野先生的角度看，他已经杳无消息。最后


Prompt template的支持

In [20]:
from llama_index import QuestionAnswerPrompt

# load documents
new_documents = SimpleDirectoryReader('./data/mr_fujino').load_data()

# define custom QuestionAnswerPrompt
query_str = "请问你们海南能发货吗？"
QA_PROMPT_TMPL = (
    "请参考如下上下文信息. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "基于这样的上下文信息，请回答: {query_str}\n"
)
QA_PROMPT = QuestionAnswerPrompt(QA_PROMPT_TMPL)
# Build GPTVectorStoreIndex
index = GPTVectorStoreIndex.from_documents(new_documents, service_context=service_context)

query_engine = index.as_query_engine(
    text_qa_template=QA_PROMPT
)
response = query_engine.query(query_str)
print(response)

Batches: 100%|██████████| 1/1 [00:00<00:00, 23.92it/s]

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 7055 tokens
> [build_index_from_nodes] Total embedding token usage: 7055 tokens



Batches: 100%|██████████| 1/1 [00:00<00:00, 217.24it/s]

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 24 tokens
> [retrieve] Total embedding token usage: 24 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2122 tokens
> [get_response] Total LLM token usage: 2122 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens
根据上下文信息，可以看出作者在谈论自己的旅行经历和在日本的经历。在藤野先生的段落中，作者提到了东京的樱花烂熳，但同时也提到了留学生的速成班。因此，可以推断藤野先生所在的段落讨论的并非樱花，而是留学生。此外，在藤野先生的段落中，作者提到了仙台医学专门学校，并在此讲述了自己的旅行经历。因此，可以得出结论，作者并没有提到海南或任何与海南相关的事物，因此，回答是：不能。
